In [1]:
# GRU model
import pandas as pd
import numpy as np
from dataset import LABELS
train = pd.read_pickle('features_training.pkl')
val = pd.read_pickle('features_validation.pkl')

X_train = np.array([x[0] for x in train])
y_train = np.array([x[1] for x in train])
X_val = np.array([x[0] for x in val])
y_val = np.array([x[1] for x in val])

from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the label encoder using your labels (combine both train and val labels if they have different classes)
label_encoder.fit(np.concatenate((y_train, y_val)))

# Transform your string labels to integer labels for both training and validation sets
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (MultiHeadAttention, Dropout, LayerNormalization, Dense, TimeDistributed,
                                     BatchNormalization, Conv1D, MaxPooling1D, Flatten, GlobalMaxPooling1D)
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

results = pd.DataFrame(columns=['num_heads', 'num_layers', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, dropout_rate):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(embed_dim, activation='relu'),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, inputs):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)

for num_heads in [2, 4, 8]:
    for num_layers in [1, 2, 3]:
        for dropout_rate in [0.2, 0.4, 0.6]:
            for batch_size in [32, 64, 128]:
                for epochs in [10, 20, 30]:
                    num_classes = len(y_train_encoded)  # Number of unique classes in your dataset
                    input_shape = (39, 44)
                    embed_dim = 128

                    inputs = tf.keras.Input(shape=input_shape)
                    x = Conv1D(filters=128, kernel_size=3, activation='relu')(inputs)
                    x = BatchNormalization()(x)
                    x = MaxPooling1D(pool_size=2)(x)
                    x = Dropout(dropout_rate)(x)

                    for _ in range(num_layers):
                        x = TransformerBlock(embed_dim, num_heads, dropout_rate)(x)

                    x = GlobalMaxPooling1D()(x)
                    x = Dense(512, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(256, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    x = Dense(128, activation='relu')(x)
                    x = BatchNormalization()(x)
                    x = Dropout(dropout_rate)(x)
                    outputs = Dense(num_classes, activation='softmax')(x)

                    model = tf.keras.Model(inputs=inputs, outputs=outputs)

                    # Compile the model
                    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                    # Train the model with your training set and validate it with your validation set
                    history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded), verbose=2)
                                    # Record the results for the current configuration
                    row = {
                        'num_heads': num_heads,
                        'num_layers': num_layers,
                        'dropout_rate': dropout_rate,
                        'epoch': epochs,
                        'batch': batch_size,
                        'loss': history.history['loss'][-1],
                        'loss_max': np.max(history.history['loss']),
                        'accuracy': history.history['accuracy'][-1],
                        'accuracy_max': np.max(history.history['accuracy']),
                        'val_loss': history.history['val_loss'][-1],
                        'val_loss_max': np.max(history.history['val_loss']),
                        'val_accuracy': history.history['val_accuracy'][-1],
                        'val_accuracy_max': np.max(history.history['val_accuracy']),
                    }

                    results = results.append(row, ignore_index=True)


Epoch 1/10
214/214 - 14s - loss: 6.4302 - accuracy: 0.1402 - val_loss: 3.3709 - val_accuracy: 0.1958 - 14s/epoch - 67ms/step
Epoch 2/10
214/214 - 9s - loss: 2.4248 - accuracy: 0.3233 - val_loss: 1.8738 - val_accuracy: 0.4540 - 9s/epoch - 40ms/step
Epoch 3/10
214/214 - 8s - loss: 1.9811 - accuracy: 0.4230 - val_loss: 1.6633 - val_accuracy: 0.5028 - 8s/epoch - 38ms/step
Epoch 4/10
214/214 - 8s - loss: 1.7598 - accuracy: 0.4759 - val_loss: 1.4195 - val_accuracy: 0.5750 - 8s/epoch - 39ms/step
Epoch 5/10
214/214 - 9s - loss: 1.6374 - accuracy: 0.5141 - val_loss: 1.4005 - val_accuracy: 0.5897 - 9s/epoch - 40ms/step
Epoch 6/10
214/214 - 8s - loss: 1.5569 - accuracy: 0.5331 - val_loss: 1.4077 - val_accuracy: 0.5758 - 8s/epoch - 35ms/step
Epoch 7/10
214/214 - 8s - loss: 1.4789 - accuracy: 0.5595 - val_loss: 1.3765 - val_accuracy: 0.5903 - 8s/epoch - 39ms/step
Epoch 8/10
214/214 - 8s - loss: 1.3995 - accuracy: 0.5754 - val_loss: 1.3111 - val_accuracy: 0.6108 - 8s/epoch - 39ms/step
Epoch 9/10
214

C:\Users\mikol\AppData\Local\Temp\ipykernel_7372\2835871655.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/20
214/214 - 20s - loss: 6.4879 - accuracy: 0.1036 - val_loss: 2.9611 - val_accuracy: 0.2146 - 20s/epoch - 95ms/step
Epoch 2/20
214/214 - 9s - loss: 2.5297 - accuracy: 0.2970 - val_loss: 2.0912 - val_accuracy: 0.3903 - 9s/epoch - 42ms/step
Epoch 3/20
214/214 - 9s - loss: 2.0467 - accuracy: 0.4020 - val_loss: 1.6879 - val_accuracy: 0.4979 - 9s/epoch - 40ms/step
Epoch 4/20
214/214 - 8s - loss: 1.8039 - accuracy: 0.4653 - val_loss: 1.7434 - val_accuracy: 0.4831 - 8s/epoch - 39ms/step
Epoch 5/20
214/214 - 9s - loss: 1.6802 - accuracy: 0.4996 - val_loss: 1.6784 - val_accuracy: 0.5147 - 9s/epoch - 41ms/step
Epoch 6/20
214/214 - 8s - loss: 1.5595 - accuracy: 0.5416 - val_loss: 1.4830 - val_accuracy: 0.5560 - 8s/epoch - 36ms/step
Epoch 7/20
214/214 - 8s - loss: 1.4913 - accuracy: 0.5576 - val_loss: 1.3892 - val_accuracy: 0.5880 - 8s/epoch - 38ms/step
Epoch 8/20
214/214 - 8s - loss: 1.3985 - accuracy: 0.5804 - val_loss: 1.2607 - val_accuracy: 0.6209 - 8s/epoch - 39ms/step
Epoch 9/20
214

C:\Users\mikol\AppData\Local\Temp\ipykernel_7372\2835871655.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/30
214/214 - 18s - loss: 6.4275 - accuracy: 0.1424 - val_loss: 3.3524 - val_accuracy: 0.2164 - 18s/epoch - 86ms/step
Epoch 2/30
214/214 - 9s - loss: 2.4255 - accuracy: 0.3361 - val_loss: 1.9823 - val_accuracy: 0.4404 - 9s/epoch - 41ms/step
Epoch 3/30
214/214 - 8s - loss: 1.9356 - accuracy: 0.4380 - val_loss: 1.7936 - val_accuracy: 0.4743 - 8s/epoch - 39ms/step
Epoch 4/30
214/214 - 8s - loss: 1.7463 - accuracy: 0.4843 - val_loss: 1.7224 - val_accuracy: 0.4773 - 8s/epoch - 38ms/step
Epoch 5/30
214/214 - 9s - loss: 1.6071 - accuracy: 0.5176 - val_loss: 1.4617 - val_accuracy: 0.5634 - 9s/epoch - 40ms/step
Epoch 6/30
214/214 - 8s - loss: 1.5174 - accuracy: 0.5508 - val_loss: 1.3200 - val_accuracy: 0.5996 - 8s/epoch - 38ms/step
Epoch 7/30
214/214 - 8s - loss: 1.4559 - accuracy: 0.5646 - val_loss: 1.3231 - val_accuracy: 0.6014 - 8s/epoch - 38ms/step
Epoch 8/30
214/214 - 9s - loss: 1.3885 - accuracy: 0.5881 - val_loss: 1.1966 - val_accuracy: 0.6397 - 9s/epoch - 40ms/step
Epoch 9/30
214

...

Epoch 1/30
54/54 - 25s - loss: 8.5580 - accuracy: 0.0705 - val_loss: 8.4180 - val_accuracy: 0.0462 - 25s/epoch - 467ms/step
Epoch 2/30
54/54 - 16s - loss: 6.1749 - accuracy: 0.1642 - val_loss: 6.2464 - val_accuracy: 0.1456 - 16s/epoch - 299ms/step
Epoch 3/30
54/54 - 16s - loss: 3.1872 - accuracy: 0.2784 - val_loss: 3.9716 - val_accuracy: 0.2576 - 16s/epoch - 297ms/step
Epoch 4/30
54/54 - 16s - loss: 2.3025 - accuracy: 0.3763 - val_loss: 2.4802 - val_accuracy: 0.3372 - 16s/epoch - 296ms/step
Epoch 5/30
54/54 - 16s - loss: 1.9627 - accuracy: 0.4357 - val_loss: 1.7992 - val_accuracy: 0.4932 - 16s/epoch - 299ms/step
Epoch 6/30
54/54 - 16s - loss: 1.7590 - accuracy: 0.4849 - val_loss: 1.6950 - val_accuracy: 0.5282 - 16s/epoch - 292ms/step
Epoch 7/30
54/54 - 16s - loss: 1.6073 - accuracy: 0.5266 - val_loss: 1.6385 - val_accuracy: 0.5457 - 16s/epoch - 298ms/step
Epoch 8/30
54/54 - 16s - loss: 1.4576 - accuracy: 0.5655 - val_loss: 1.6652 - val_accuracy: 0.5577 - 16s/epoch - 303ms/step
Epoch 9/

C:\Users\mikol\AppData\Local\Temp\ipykernel_7372\2835871655.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/10
214/214 - 31s - loss: 7.5776 - accuracy: 0.0260 - val_loss: 5.4721 - val_accuracy: 0.0365 - 31s/epoch - 145ms/step
Epoch 2/10
214/214 - 21s - loss: 3.6596 - accuracy: 0.0407 - val_loss: 3.8553 - val_accuracy: 0.0338 - 21s/epoch - 100ms/step
Epoch 3/10
214/214 - 20s - loss: 3.4585 - accuracy: 0.0367 - val_loss: 4.0061 - val_accuracy: 0.0382 - 20s/epoch - 95ms/step
Epoch 4/10
214/214 - 20s - loss: 3.4283 - accuracy: 0.0391 - val_loss: 4.1258 - val_accuracy: 0.0357 - 20s/epoch - 94ms/step
Epoch 5/10
214/214 - 21s - loss: 3.4097 - accuracy: 0.0462 - val_loss: 4.5288 - val_accuracy: 0.0544 - 21s/epoch - 100ms/step
Epoch 6/10
214/214 - 21s - loss: 3.3201 - accuracy: 0.0711 - val_loss: 3.7909 - val_accuracy: 0.0627 - 21s/epoch - 97ms/step
Epoch 7/10
214/214 - 21s - loss: 3.1585 - accuracy: 0.0941 - val_loss: 3.5522 - val_accuracy: 0.0674 - 21s/epoch - 97ms/step
Epoch 8/10
214/214 - 21s - loss: 3.1419 - accuracy: 0.0951 - val_loss: 3.2540 - val_accuracy: 0.0914 - 21s/epoch - 99ms/st

C:\Users\mikol\AppData\Local\Temp\ipykernel_7372\2835871655.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(row, ignore_index=True)


Epoch 1/20
214/214 - 32s - loss: 7.5393 - accuracy: 0.0380 - val_loss: 4.6996 - val_accuracy: 0.0440 - 32s/epoch - 149ms/step
Epoch 2/20
214/214 - 23s - loss: 3.6033 - accuracy: 0.0560 - val_loss: 3.6512 - val_accuracy: 0.0562 - 23s/epoch - 107ms/step
Epoch 3/20
214/214 - 23s - loss: 3.3469 - accuracy: 0.0689 - val_loss: 3.5102 - val_accuracy: 0.0580 - 23s/epoch - 106ms/step
Epoch 4/20
214/214 - 22s - loss: 3.2046 - accuracy: 0.0838 - val_loss: 3.7588 - val_accuracy: 0.0550 - 22s/epoch - 105ms/step
Epoch 5/20
214/214 - 23s - loss: 3.1141 - accuracy: 0.0970 - val_loss: 4.1244 - val_accuracy: 0.0419 - 23s/epoch - 106ms/step
Epoch 6/20
214/214 - 22s - loss: 3.0169 - accuracy: 0.1230 - val_loss: 3.4114 - val_accuracy: 0.0671 - 22s/epoch - 102ms/step
Epoch 7/20
214/214 - 22s - loss: 2.9024 - accuracy: 0.1466 - val_loss: 3.3176 - val_accuracy: 0.0834 - 22s/epoch - 102ms/step
Epoch 8/20
214/214 - 22s - loss: 2.8194 - accuracy: 0.1665 - val_loss: 3.3338 - val_accuracy: 0.0939 - 22s/epoch - 105

In [ ]:
results.to_pickle('model_transformer')